In [24]:
%pip install -U langchain-community
%pip install pymupdf docarray

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 KB 2.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 KB 12.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("./kubernetes-cheatsheet.pdf")
pages = loader.load()

In [22]:
pages[0]

Document(metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20250518155345', 'source': './kubernetes-cheatsheet.pdf', 'file_path': './kubernetes-cheatsheet.pdf', 'total_pages': 1, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': 'D:20250518155345', 'page': 0}, page_content="Kubernetes Cheatsheet\nWhat is Kubernetes \nKapsule and \nKubernetes Kosmos?\nKubernetes is an open-source platform \nthat enables developers to manage their \ncontainerized applications. Kapsule and \nKosmo both provide a managed \nenvironment for creating, configuring,\nand running clusters of pre-configured \nmachines.\u2028\nKapsule clusters are composed solely of \nScaleway Instances whereas Kosmos is a \nmanaged Multi-Cloud Kubernetes Engine \nthat allows you to connect Instances and \nvirtual or dedicated servers from any \ncloud provider to a single managed \nControl-Plane.\nViewing and finding \nr

In [12]:
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="llama3")

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
index_creator = VectorstoreIndexCreator( # 一鍵建立 index + retriever
    vectorstore_cls=DocArrayInMemorySearch, # 	記憶體內向量資料庫
    embedding=embeddings)
index = index_creator.from_documents(pages)

/home/itachi/.local/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [26]:
query ="how to use Kubernetes LOGS  ?"

In [27]:
from langchain_ollama.llms import OllamaLLM
chat = OllamaLLM(temperature=0.0, model="llama3")

In [28]:
response = index.query(query, llm = chat)

In [29]:
from IPython.display import display, Markdown
display(Markdown(response))

To use Kubernetes logs, you can use the `kubectl logs` command. This command allows you to view the logs of a specific pod or node.

Here are some examples of how to use `kubectl logs`:

* To view the logs of a specific pod:
```
kubectl logs [pod_name]
```
This will display the logs for the specified pod.

* To view the logs of a specific node:
```
kubectl logs [node_name] -n [namespace]
```
This will display the logs for the specified node in the specified namespace.

* To follow the logs of a specific pod or node:
```
kubectl logs [pod_name] --follow
```
This will allow you to view the logs as they are generated, rather than just displaying the existing logs.

* To view the logs of all pods in a namespace:
```
kubectl logs -n [namespace]
```
This will display the logs for all pods in the specified namespace.

Note: You can also use `kubectl logs` with other options to customize your log viewing experience. For example, you can specify a specific container or log level using the `-c` and `-l` options respectively.

In [30]:
db = DocArrayInMemorySearch.from_documents(
    pages, 
    embeddings
)
retriever = db.as_retriever()

In [31]:
from langchain.prompts import ChatPromptTemplate
system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [32]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
question_answer_chain = create_stuff_documents_chain(chat, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

chain.invoke({"input": query})

{'input': 'how to use Kubernetes LOGS  ?',
 'context': [Document(metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20250518155345', 'source': './kubernetes-cheatsheet.pdf', 'file_path': './kubernetes-cheatsheet.pdf', 'total_pages': 1, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': 'D:20250518155345', 'page': 0}, page_content="Kubernetes Cheatsheet\nWhat is Kubernetes \nKapsule and \nKubernetes Kosmos?\nKubernetes is an open-source platform \nthat enables developers to manage their \ncontainerized applications. Kapsule and \nKosmo both provide a managed \nenvironment for creating, configuring,\nand running clusters of pre-configured \nmachines.\u2028\nKapsule clusters are composed solely of \nScaleway Instances whereas Kosmos is a \nmanaged Multi-Cloud Kubernetes Engine \nthat allows you to connect Instances and \nvirtual or dedicated servers from any \ncloud provider to a 

In [33]:
chain.invoke({"input": "How to get Database"})

{'input': 'How to get Database',
 'context': [Document(metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20250518155345', 'source': './kubernetes-cheatsheet.pdf', 'file_path': './kubernetes-cheatsheet.pdf', 'total_pages': 1, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': 'D:20250518155345', 'page': 0}, page_content="Kubernetes Cheatsheet\nWhat is Kubernetes \nKapsule and \nKubernetes Kosmos?\nKubernetes is an open-source platform \nthat enables developers to manage their \ncontainerized applications. Kapsule and \nKosmo both provide a managed \nenvironment for creating, configuring,\nand running clusters of pre-configured \nmachines.\u2028\nKapsule clusters are composed solely of \nScaleway Instances whereas Kosmos is a \nmanaged Multi-Cloud Kubernetes Engine \nthat allows you to connect Instances and \nvirtual or dedicated servers from any \ncloud provider to a single man